# Embeddings

In [1]:
import os, sys

import numpy as np
import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from metrics import Metrics
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction, TfidfFeatureExtraction, SentenceTransformerFeatureExtraction, BertFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Read csv files and load as df

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
predictions_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/pr

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [4]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_1-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_1-observation/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_2-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_2-observation/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [5]:
col_name = 'Base Sentence'
predictions = DataProcessing.df_to_list(predictions_df, col_name)
observations = DataProcessing.df_to_list(observations_df, col_name)
len(predictions), len(observations)

(694, 1891)

## Extract Embeddings with Spacy

In [6]:
by_prediction_df = pd.read_csv('../data/entailment/entailment-v1.csv')  
by_prediction_df

,Prediction,Observation,Entailment Label
0,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","In Q2 2025, a financial research advisor envisions that the research and development expenses at Google has some probability to remain stable.",NEUTRAL
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to the financial top executive at BlackRock, the stock price at Amazon may rise in Q4 2028.",NEUTRAL
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"In the fourth quarter of 2027, Wells Fargo envisions that the operating cash flow at Intel has some probability to remain stable.",NEUTRAL
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On March 1, 2029, the financial advisor at Wells Fargo envisions that the inflation rate at the Federal Reserve has some probability to remain stable.",NEUTRAL
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",NEUTRAL
6,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",NEUTRAL
7,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",NEUTRAL
8,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Apple stock price decreased in August 2024, according to Roger.",NEUTRAL
9,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL


In [7]:
by_prediction_df

,Prediction,Observation,Entailment Label
0,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","In Q2 2025, a financial research advisor envisions that the research and development expenses at Google has some probability to remain stable.",NEUTRAL
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to the financial top executive at BlackRock, the stock price at Amazon may rise in Q4 2028.",NEUTRAL
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"In the fourth quarter of 2027, Wells Fargo envisions that the operating cash flow at Intel has some probability to remain stable.",NEUTRAL
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On March 1, 2029, the financial advisor at Wells Fargo envisions that the inflation rate at the Federal Reserve has some probability to remain stable.",NEUTRAL
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",NEUTRAL
6,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",NEUTRAL
7,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",NEUTRAL
8,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Apple stock price decreased in August 2024, according to Roger.",NEUTRAL
9,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL


In [8]:
disable_components = [""]
# pred_spacy_fe = SpacyFeatureExtraction(predictions_df, "Base Sentence")
spacy_fe_pred = SpacyFeatureExtraction(by_prediction_df, 'Prediction')
spacy_embeddings_pred = spacy_fe_pred.sentence_feature_extraction()

spacy_fe_obser = SpacyFeatureExtraction(by_prediction_df, 'Observation')
obser_embeddings_pred = spacy_fe_obser.sentence_feature_extraction()

spacy_cs_metrics = Metrics.get_cosine_similarity(spacy_embeddings_pred, obser_embeddings_pred)
by_prediction_df['Spacy'] = spacy_cs_metrics
by_prediction_df

100%|██████████| 12/12 [00:00<00:00, 5379.04it/s]


,Prediction,Observation,Entailment Label,Spacy
0,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","In Q2 2025, a financial research advisor envisions that the research and development expenses at Google has some probability to remain stable.",NEUTRAL,0.773352
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to the financial top executive at BlackRock, the stock price at Amazon may rise in Q4 2028.",NEUTRAL,0.866826
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"In the fourth quarter of 2027, Wells Fargo envisions that the operating cash flow at Intel has some probability to remain stable.",NEUTRAL,0.865429
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL,0.812578
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On March 1, 2029, the financial advisor at Wells Fargo envisions that the inflation rate at the Federal Reserve has some probability to remain stable.",NEUTRAL,0.845318
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",NEUTRAL,0.815297
6,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",NEUTRAL,0.859226
7,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",NEUTRAL,0.927874
8,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Apple stock price decreased in August 2024, according to Roger.",NEUTRAL,0.876008
9,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL,0.900117


## Extract Embeddings with Sentence Transformer

In [9]:
st_fe_pred = SentenceTransformerFeatureExtraction(by_prediction_df, 'Prediction')
st_embeddings_pred = st_fe_pred.sentence_feature_extraction()

st_fe_obser = SentenceTransformerFeatureExtraction(by_prediction_df, 'Observation')
st_embeddings_obser = st_fe_obser.sentence_feature_extraction()

st_cs_metrics = Metrics.get_cosine_similarity(st_embeddings_pred, st_embeddings_obser)
by_prediction_df['SentenceTransformer'] = st_cs_metrics
by_prediction_df

  0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 4325.88it/s]


,Prediction,Observation,Entailment Label,Spacy,SentenceTransformer
0,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","In Q2 2025, a financial research advisor envisions that the research and development expenses at Google has some probability to remain stable.",NEUTRAL,0.773352,0.328824
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to the financial top executive at BlackRock, the stock price at Amazon may rise in Q4 2028.",NEUTRAL,0.866826,0.644740
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"In the fourth quarter of 2027, Wells Fargo envisions that the operating cash flow at Intel has some probability to remain stable.",NEUTRAL,0.865429,0.311582
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL,0.812578,0.321734
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On March 1, 2029, the financial advisor at Wells Fargo envisions that the inflation rate at the Federal Reserve has some probability to remain stable.",NEUTRAL,0.845318,0.282765
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",NEUTRAL,0.815297,0.490033
6,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",NEUTRAL,0.859226,0.489393
7,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",NEUTRAL,0.927874,0.771568
8,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Apple stock price decreased in August 2024, according to Roger.",NEUTRAL,0.876008,0.375553
9,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL,0.900117,0.394057


## Extract Embeddings with Bert 

In [10]:
bert_fe_pred = BertFeatureExtraction(by_prediction_df, 'Prediction')
bert_embeddings_pred = bert_fe_pred.sentence_feature_extraction()

bert_fe_obser = BertFeatureExtraction(by_prediction_df, 'Observation')
bert_embeddings_obser = bert_fe_obser.sentence_feature_extraction()

bert_cs_metrics = Metrics.get_cosine_similarity(bert_embeddings_pred, bert_embeddings_obser)
by_prediction_df['BERT'] = bert_cs_metrics
by_prediction_df

100%|██████████| 12/12 [00:00<00:00, 6845.05it/s]


,Prediction,Observation,Entailment Label,Spacy,SentenceTransformer,BERT
0,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","In Q2 2025, a financial research advisor envisions that the research and development expenses at Google has some probability to remain stable.",NEUTRAL,0.773352,0.328824,0.899819
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to the financial top executive at BlackRock, the stock price at Amazon may rise in Q4 2028.",NEUTRAL,0.866826,0.644740,0.941268
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"In the fourth quarter of 2027, Wells Fargo envisions that the operating cash flow at Intel has some probability to remain stable.",NEUTRAL,0.865429,0.311582,0.896069
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL,0.812578,0.321734,0.903572
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On March 1, 2029, the financial advisor at Wells Fargo envisions that the inflation rate at the Federal Reserve has some probability to remain stable.",NEUTRAL,0.845318,0.282765,0.819832
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",NEUTRAL,0.815297,0.490033,0.913402
6,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",NEUTRAL,0.859226,0.489393,0.918903
7,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",NEUTRAL,0.927874,0.771568,0.918560
8,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Apple stock price decreased in August 2024, according to Roger.",NEUTRAL,0.876008,0.375553,0.888524
9,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL,0.900117,0.394057,0.874122


## Extract Embeddings with TF x IDF

In [11]:
# tfidf_fe_pred = TfidfFeatureExtraction(by_prediction_df, 'Prediction')
# tfidf_pred_df = tfidf_fe_pred.feature_scores(max_features=300)
# tfidf_pred_df[:1]
# tfidf_embedding_pred = tfidf_pred_df.iloc[:1 , 2:].to_numpy()

# tfidf_fe_obser = TfidfFeatureExtraction(by_prediction_df, 'Observation')
# tfidf_obser_df = tfidf_fe_obser.feature_scores(max_features=300)
# tfidf_obser_df[:1]
# tfidf_embedding_obser = tfidf_obser_df.iloc[:1 , 2:].to_numpy()

# tfidf_cs_metrics = Metrics.get_cosine_similarity(tfidf_embedding_pred, tfidf_embedding_obser)
# by_prediction_df['TF x IDF'] = tfidf_cs_metrics
# by_prediction_df

In [12]:
pred_obser_features_df = pd.pivot_table(by_prediction_df, values='Entailment Label', index=['Prediction', 'Observation', 'Entailment Label', 'Spacy', 'SentenceTransformer', 'BERT'])
pred_obser_features_df

Empty DataFrame
Columns: []
Index: [(Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise., In Q2 2025, a financial research advisor envisions that the research and development expenses at Google has some probability to remain stable., NEUTRAL, 0.7733518481254578, 0.3288244605064392, 0.8998185396194458), (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027., According to the financial top executive at BlackRock, the stock price at Amazon may rise in Q4 2028., NEUTRAL, 0.8668255805969238, 0.6447398662567139, 0.9412680864334106), (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027., In the fourth quarter of 2027, Wells Fargo envisions that the operating cash flow at Intel has some probability to remain stable., NEUTRAL, 0.8654288053512573, 0.31158241629600525, 0.8960685729980469), (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027., Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University., NEUTRAL, 0.8125780820846558, 0.321733683347702, 0.9035715460777283), (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027., On March 1, 2029, the financial advisor at Wells Fargo envisions that the inflation rate at the Federal Reserve has some probability to remain stable., NEUTRAL, 0.8453179597854614, 0.2827652394771576, 0.8198323249816895), (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027., On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase., NEUTRAL, 0.8152968883514404, 0.4900325536727905, 0.9134016633033752), (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027., The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase., NEUTRAL, 0.8592255115509033, 0.4893929362297058, 0.9189028739929199), (On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase., According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026., NEUTRAL, 0.9278740882873535, 0.7715675234794617, 0.9185597896575928), (On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase., Apple stock price decreased in August 2024, according to Roger., NEUTRAL, 0.8760083913803101, 0.37555310130119324, 0.8885236978530884), (On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase., Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University., NEUTRAL, 0.9001173377037048, 0.3940570652484894, 0.8741218447685242), (On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase., The trading volume at Alibaba remained stable, according to the research advisor at Bank of America., NEUTRAL, 0.8823338747024536, 0.15972918272018433, 0.8424957990646362), (On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase., The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase., NEUTRAL, 0.8905181884765625, 0.40385186672210693, 0.8800209760665894)]